In [4]:
import numpy as np
from numpy import dot, sum, zeros
from numpy.linalg import inv, eig
from numpy.random import normal


def g(lamb,X,Q):
    A = dot(X.T, X) - lamb * Q
    A_inv = inv(A)
    G = lamb**2 * dot(A_inv,A_inv)
    return G

def expected_g(lamb,n,d,Q,numSimuls):
    expected_G_n = zeros((d,d))
    for sim in range(numSimuls):
        X_n = normal(0,1,size=(n,d))
        G_n = g(lamb,X_n,Q)
        expected_G_n = expected_G_n + G_n/numSimuls
    return expected_G_n

def dg_approx(lamb,X,Q,eps=10**(-7)):
    G1 = g(lamb,X,Q)
    G2 = g(lamb+eps,X,Q)
    dG = (G2 - G1)/eps
    return dG

def dg(lamb,X,Q):
    XTX = dot(X.T,X)
    A = XTX - lamb * Q
    A_inv = inv(A)
    A_inv_2 = dot(A_inv,A_inv)

    dG_lamb1 = 2*lamb * A_inv_2

    B = A_inv
    B_prime = -dot(dot(B, Q), B)
    res = dot(B,B_prime) + dot(B_prime,B)
    dG_lamb2 = lamb**2 * res
    dG_lamb = dG_lamb1 + dG_lamb2
    return -dG_lamb

def expected_dg(lamb,n,d,Q,numSimuls,approx=False):
    d_expected_G_n = zeros((d,d))
    for sim in range(numSimuls):
        X_n = normal(0,1,size=(n,d))
        dG_n = None
        if approx:
            dG_n = dg_approx(lamb,X_n,Q)
        else:
            dG_n = dg(lamb,X_n,Q)
        d_expected_G_n = d_expected_G_n + dG_n/numSimuls
    return d_expected_G_n


def h(lamb,X,Q):
    mat = dot(X.T, X) - lamb * Q
    mat_inv = inv(mat)
    H = sum(dot(mat_inv, X.T)**2)
    return H

def expected_h(lamb,n,d,Q,numSimuls):
    expected_H_n = 0
    for sim in range(numSimuls):
        X_n = normal(0,1,size=(n,d))
        H_n = h(lamb,X_n,Q)
        expected_H_n = expected_H_n + H_n/numSimuls
    return expected_H_n

def dh_approx(lamb,X,Q,eps=10**(-7)):
    H1 = h(lamb,X,Q)
    H2 = h(lamb+eps,X,Q)
    dH = (H2 - H1)/eps
    return dH

def dh(lamb,X,Q):
    XTX = dot(X.T,X)
    A = XTX - lamb * Q
    A_inv = inv(A)
    B = A_inv
    B_prime = -dot(dot(B,Q),B)
    res1 = dot(X,B_prime)
    res2 = 2 * np.dot(X,B)
    dh_dlamb = sum(res1 * res2)
    return -dh_dlamb

def expected_dh(lamb,n,d,Q,numSimuls,approx=False):
    d_expected_H_n = 0
    for sim in range(numSimuls):
        X_n = normal(0,1,size=(n,d))
        if approx:
            dH_n = dh_approx(lamb,X_n,Q)
        else:
            dH_n = dh(lamb,X_n,Q)
        d_expected_H_n = d_expected_H_n + dH_n/numSimuls
    return d_expected_H_n


def isPSD(mat):
    eigs = eig(mat)[0]
    print(eigs)
    for i in range(len(eigs)):
        if eigs[i] < 0: return False
    return True


In [22]:
n = 10
d = n
numSimuls = 100

lamb = 500
Q = [i/d for i in range(1,d+1)] * np.identity(d)

expected_G_n1 = expected_g(lamb,n+1,d,Q,numSimuls)
expected_G_n = expected_g(lamb,n,d,Q,numSimuls)
d_expected_G_n = expected_g(lamb,n,d,Q,numSimuls)

expected_H_n1 = expected_h(lamb,n+1,d,Q,numSimuls)
expected_H_n = expected_h(lamb,n,d,Q,numSimuls)
d_expected_H_n = expected_dh(lamb,n,d,Q,numSimuls)

mat = (expected_G_n - expected_G_n1) * d_expected_H_n - (expected_H_n - expected_H_n1) * d_expected_G_n
res = isPSD(mat)
print(res)


# Debugging
# print(expected_G_n1)
# print(expected_G_n)
# print(d_expected_G_n)
# print(expected_H_n1)
# print(expected_H_n)
# print(d_expected_H_n)

[0.41014781 0.07613795 0.03129533 0.01712459 0.01073029 0.0073375
 0.00532507 0.00254808 0.00405606 0.0031896 ]
True


In [31]:
# Numerical gradient checking
n = 100
d = 50

lamb = 8
Q = [i for i in range(1,d+1)] * np.identity(d)
Xa = normal(size=(n,d))
eps = 10**(-5)

print(dg_approx(lamb,Xa,Q,eps) / dg(lamb,Xa,Q)) #if correct, entries should be close to 1
print(dh_approx(lamb,Xa,Q,eps) / dh(lamb,Xa,Q)) #if correct, should be close to 1

[[0.99550989 0.99551023 0.99550987 ... 0.99551021 0.99551013 0.99551032]
 [0.99551023 0.99550991 0.99551021 ... 0.9955101  0.99550988 0.99551019]
 [0.99550987 0.99551021 0.99550473 ... 0.99551016 0.99551053 0.99551211]
 ...
 [0.99551021 0.9955101  0.99551016 ... 0.99551001 0.99551007 0.99551012]
 [0.99551013 0.99550988 0.99551053 ... 0.99551007 0.99550906 0.99551019]
 [0.99551032 0.99551019 0.99551211 ... 0.99551012 0.99551019 0.99550915]]
0.9966071069899405
